In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


In [0]:
# Initialize Spark session
spark

# Install the required packages
%pip install langchain-community langchain beautifulsoup4 pypdf langchain-openai chromadb faiss-cpu sentence-transformers


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.

In [0]:
dbutils.library.restartPython()

In [0]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader

# Ensure the file path is correct
file_path = '/Volumes/workspace/default/tempstorage/jp_morgan_weekly_market_recap.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()
docs

[Document(metadata={'producer': 'iText 2.1.3 (by lowagie.com)', 'creator': 'PyPDF', 'creationdate': '2025-11-30T19:47:15-06:00', 'moddate': '2025-11-30T19:47:15-06:00', 'title': 'Weekly Market Recap', 'source': '/Volumes/workspace/default/tempstorage/jp_morgan_weekly_market_recap.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Not FDIC Insured | No Bank Guarantee | May Lose Value\nMarket Insights\nU.S. | December 1, 2025\nWeekly Market Recap\nTheweekinreview\n\x95 Retail sales rose 0.2% m/m and 4.3% y/y\n\x95 Consumer confidence fell to 88.7 from 95.5\nTheweekahead\n\x95 PMIs\n\x95 Consumer spending\n\x95 PCE price index\nThoughtoftheweek\nLast Friday, U.S. consumers, hungry for deals instead of\nThanksgiving food, kicked off what is expected to be a solid\nholiday shopping season. A recent National Retail Federa-\ntion survey suggests this Thanksgiving shopping weekend\ncould have seen a record 187mn shoppers, with holiday\nretail sales expected to surpass $1tn fo

In [0]:
%pip install --upgrade langchain langchain-community

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
pip install langchain-text-splitters


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

documents = text_splitter.split_documents(docs)


In [0]:
len(documents)

20

In [0]:
## Vector Embedding And Vector Store
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
db = Chroma.from_documents(documents, embeddings)

/home/spark-ebc0c31e-0b7a-4aca-af3a-c5/.ipykernel/10560/command-4591373645843690-3237889524:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [0]:
%pip install transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
 dbutils.library.restartPython()

In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model_name =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

In [0]:
def get_llm_answer(query):
    # 1. RAG: retrieve relevant chunks
    retriever_results = db.similarity_search(query, k=4)
    reference_data = "\n\n".join([doc.page_content for doc in retriever_results])

    # 2. Construct prompt for the local Mistral model
    prompt = f"""
You are a helpful AI assistant. Answer based ONLY on the reference context.
Give short, precise answers in bullet points with emojis.
If the answer is not in the context, say: "Information not found in the document."

QUESTION:
{query}

REFERENCE CONTEXT:
{reference_data}

ANSWER:
"""

    # 3. Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 4. Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.4,
        do_sample=True,
        top_p=0.9,
    )

    # 5. Decode output (remove prompt)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    final_answer = answer.replace(prompt, "").strip()

    return final_answer


In [0]:
import torch
torch.cuda.is_available(), torch.cuda.device_count()


(False, 0)

In [0]:
prompt = "Say hi"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Say hi to my little friend!

Verse 2:
I'm a little bird,


In [0]:
response = get_llm_answer("What were the key economic indicators mentioned this week?")
print(response)


The key economic indicators mentioned this week include:

1. 30 Year Treasury, FactSet; 10 Year German Bund, FactSet.
2. 3 Month LIBOR, British Bankers' Association; 3 Month EURIBOR, European Banking Federation; 6 Month CD, Federal Reserve; 30 Year Mortgage, Mortgage Bankers Association (MBA); Prime Rate: Federal Reserve.
3. Commodities: Gold, FactSet; Crude Oil (WTI), FactSet; Gasoline, FactSet; Natural Gas, FactSet; Silver, FactSet; Copper, FactSet; Corn, FactSet.
4. Banks: Dollar per Pound, FactSet; Dollar per Euro, FactSet; Yen per Dollar, FactSet.
5. S&P Index Characteristics: Dividend yield provided by FactSet Pricing database.
6. Markets: Dividend yield provided by FactSet.
7. Fwd. P/E is a bottom-up weighted harmonic averageusing First Call Mean estimates for the "Next 12 Months" (NTM) period.
8. Market cap is a bottom-up weighted average based on share information from Compustat and price information from FactSet's Pricing database as provided by Standard & Poor's.


In [0]:
response2 = get_llm_answer("Give me the top 3 sectors based on 1-week returns.")
print(response2)


1. The top 3 sectors based on 1-week returns are:
- Energy
- Consumer Staples
- Health Care

2. The top 3 sectors based on 1-month returns are:
- Energy
- Consumer Staples
- Health Care

3. The top 3 sectors based on 1-year returns are:
- Energy
- Consumer Staples
- Health Care

4. The top 3 sectors based on 1-week returns are:
- Energy
- Consumer Staples
- Health Care

5. The top 3 sectors based on 1-month returns are:
- Energy
- Consumer Staples
- Health Care

6. The top 3 sectors based on 1-year returns are:
- Energy
- Consumer Staples
- Health Care

7. The top 3 sectors based on 1-week returns are:
- Energy
- Consumer Staples
- Health Care

8. The top 3 sectors based on 1-month returns are:
- Energy
- Consumer Staples
- Health Care

9. The top 3 sectors based on 1-year returns are:
- Energy
- Consumer Staples
- Health Care

10


In [0]:
response3 = get_llm_answer("Explain me this week review in brief")
print(response3)

The Market Insights program provides comprehensive data and commentary on global markets without reference to products. The program explores the implications of current economic data and changing market conditions. The J.P. Morgan Asset Management Market Insights and Portfolio Insights programs, as non-independent research, have not been prepared in accordance with legal requirements designed to promote the independence of investment research, nor are they subject to any prohibition on dealing ahead of the dissemination of investment research. This document is a general communication being provided for informational purposes only. It is educational in nature and not designed to be taken as advice or a recommendation for any specific investment product, strategy, plan feature or other purpose in any jurisdiction, nor is it a commitment from J.P. Morgan Asset Management or any of its subsidiaries to participate in any of the transactions mentioned herein. Any examples used are generic, a